# Imports

In [1]:
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization,MaxPool2D
from keras.optimizers import Adam,SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import CategoricalCrossentropy,SparseCategoricalCrossentropy
import keras

import numpy as np
import pandas as pd 

import os
import cv2
import shutil
import glob

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
from keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split

# Damage Detection

## Data Preparation
1. For Handling Imbalanced Dataset
    * performed flips on existing class images with less quantity
    * Added new labels and images to existing dataset
    * performed stratified Split for train , 
    * created categorical encoding for multilabel classfication
    
2. For small dataset
    * performed augmentation
    * performed further image preprocesssing

In [2]:
# filename = r"C:\projects\contelligenz\damage_detect\Train Copy\790.jpeg"# change
# img = cv2.imread(filename)
# Flip_Horizontal = cv2.flip(img, 1) # 1 means Horizontal Flip
# cv2.imwrite(f"./templamp/1097.jpeg", Flip_Horizontal) # change

In [3]:
labels = pd.read_csv('./newtrainlabels.csv')
labels['class'] = labels['class'].apply(lambda x: [i.strip() for i in x.split(",")])
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from skmultilearn.model_selection import iterative_train_test_split
x = labels.drop(['class'],axis=1).values
y = labels['class']
mlb = MultiLabelBinarizer()
actuals = mlb.fit_transform(y)
x_train,y_train,x_test,y_test = iterative_train_test_split(x, actuals, test_size = 0.2)

In [4]:
mlb.classes_

array(['dent', 'glass_shatter', 'head_lamp', 'scratch', 'tail_lamp',
       'unknown'], dtype=object)

In [5]:
print("dent",np.count_nonzero(y_train[:,0])) # dent
print("glass_shatter",np.count_nonzero(y_train[:,1])) # dent
print("head_lamp",np.count_nonzero(y_train[:,2])) # dent
print("scratch",np.count_nonzero(y_train[:,3])) # dent
print("tail_lamp",np.count_nonzero(y_train[:,4])) # dent
print("unknown",np.count_nonzero(y_train[:,5])) # dent

dent 226
glass_shatter 189
head_lamp 198
scratch 213
tail_lamp 167
unknown 148


In [6]:
batch_size=32
image_size= (224, 224)
learning_rate=0.0001
path = "./datasets"
num_epochs= 100

In [7]:
train = pd.DataFrame(None)
train[['image','damage','subset']] = x_train
train['y'] = pd.Series(y_train.tolist())
train['image'] = train['image'].apply(lambda x: x.split('/')[-1])
train['y_new'] = train['y'].apply(lambda x: [i for i in mlb.inverse_transform(np.array(x,ndmin=2))[0]])

In [8]:
valid = pd.DataFrame(None)
valid[['image','damage','subset']] = x_test
valid['y'] = pd.Series(y_test.tolist())
valid['image'] = valid['image'].apply(lambda x: x.split('/')[-1])
valid['y_new'] = valid['y'].apply(lambda x: [i for i in mlb.inverse_transform(np.array(x,ndmin=2))[0]])

In [9]:
from keras.applications.vgg19 import preprocess_input
def load_dataset(dir_path,train,valid):

    img_gen1 = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        ) 

    train_ds = img_gen1.flow_from_dataframe(
        dataframe=train,
        directory=dir_path,
        x_col="image",
        y_col="y_new",
        batch_size=12,
        seed=0,
        shuffle=True,
        class_mode="categorical",
        classes=["unknown", "dent", "scratch", "glass_shatter", "head_lamp","tail_lamp"],
        target_size=image_size) # set as training data

    img_gen2 = ImageDataGenerator(rescale=1./255,
        ) 
    val_ds = img_gen2.flow_from_dataframe(
        dataframe=valid,
        directory=dir_path,
        x_col="image",
        y_col="y_new",
        batch_size=12,
        seed=0,
        shuffle=True,
        class_mode="categorical",
        classes=["unknown", "dent", "scratch", "glass_shatter", "head_lamp","tail_lamp"],
        target_size=image_size,) # set as validation data

    return train_ds, val_ds

In [10]:
train_ds, val_ds= load_dataset('./Train Copy',train.sample(train.shape[0]).reset_index(),valid.sample(valid.shape[0]).reset_index())

Found 865 validated image filenames belonging to 6 classes.
Found 216 validated image filenames belonging to 6 classes.


c:\projects\contelligenz\venv\lib\site-packages\keras\preprocessing\image.py:991: UserWarning: Found 2 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  n_invalid, x_col))


In [11]:
train_ds.class_indices

{'unknown': 0,
 'dent': 1,
 'scratch': 2,
 'glass_shatter': 3,
 'head_lamp': 4,
 'tail_lamp': 5}

In [12]:
val_ds.class_indices

{'unknown': 0,
 'dent': 1,
 'scratch': 2,
 'glass_shatter': 3,
 'head_lamp': 4,
 'tail_lamp': 5}

## Model Building

1. For model training
    * Created custom model with 14 layers.
    * used Dropouts and Regularisation.
    * used sigmoid activation for multilabel classfication

2. For model compilation
    * used adam optimizer
    * used binary loss
    * used fbeta2 for metrics for multilabel classification
    * trained for 80 epochs

3. For monitoring
    * used checkpoints

In [15]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

def model_detect(num_expressions=6):
    model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3),
    pooling='max',
    classes=num_expressions,
)
    regularizer = tf.keras.regularizers.l2(0.1)
    for layer in model.layers[:-5]:
      layer.trainable= False
    for layer in model.layers:
      if isinstance(layer, tf.keras.layers.Conv2D):
        print('Adding regularizer to layer {}'.format(layer.name))
        layer.kernel_regularizer = regularizer

    # model.get_layer('block5_conv1').trainable = True
    # model.get_layer('block5_conv2').trainable = True
    # model.get_layer('block5_conv3').trainable = True
    # model.get_layer('block5_pool').trainable = True
    flat1 = Flatten()(model.layers[-1].output)
    layer1 = Dropout(0.4)(flat1)
    layer2 = Dense(128, activation='relu', kernel_initializer='he_uniform')(layer1)
    # layer3 = Dense(64, activation='relu', kernel_initializer='he_uniform')(layer2)
    output = Dense(6, activation='sigmoid')(layer2)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    return model

def define_model(in_shape=(224, 224, 3), out_shape=6):
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu',  padding='same', input_shape=in_shape))
  model.add(Conv2D(32, (3, 3), activation='relu',  padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu',  padding='same'))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu',  padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu',  padding='same'))
  model.add(Dropout(0.2))
  model.add(Conv2D(128, (3, 3), activation='relu',  padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(out_shape, activation='sigmoid'))
  return model

In [16]:
from keras import backend
 
# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

In [17]:
exp_model = model_detect()
# exp_model = keras.models.load_model('./improvements1/weights-improvement-best_model.hdf5')
opt = Adam(learning_rate=0.0001)

# exp_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
exp_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])


filepath="./improvements1_1/weights-improvement-best_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_fbeta', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = exp_model.fit(
    
    train_ds,
    steps_per_epoch = train_ds.samples // 12,
    validation_data = val_ds, 
    validation_steps = val_ds.samples // 12,
    epochs = 80,
    callbacks=callbacks_list
    )


Adding regularizer to layer block1_conv1
Adding regularizer to layer block1_conv2
Adding regularizer to layer block2_conv1
Adding regularizer to layer block2_conv2
Adding regularizer to layer block3_conv1
Adding regularizer to layer block3_conv2
Adding regularizer to layer block3_conv3
Adding regularizer to layer block4_conv1
Adding regularizer to layer block4_conv2
Adding regularizer to layer block4_conv3
Adding regularizer to layer block5_conv1
Adding regularizer to layer block5_conv2
Adding regularizer to layer block5_conv3
Epoch 1/80
72/72 [==============================] - ETA: 0s - loss: 0.5540 - fbeta: 0.1026
Epoch 1: val_fbeta improved from -inf to 0.08131, saving model to ./improvements1_1\weights-improvement-best_model.hdf5
72/72 [==============================] - 54s 261ms/step - loss: 0.5540 - fbeta: 0.1026 - val_loss: 0.4501 - val_fbeta: 0.0813
Epoch 2/80
72/72 [==============================] - ETA: 0s - loss: 0.4314 - fbeta: 0.2786
Epoch 2: val_fbeta improved from 0.0813

KeyboardInterrupt: 

0.55415

## Predictions
1. Get the best model from checkpoint
2. Get inverse coding for label outputs
3. used 0.6 threshold for selecting labels
4. monitored fbeta score for selecting threshold

In [18]:
from keras.utils import load_img, img_to_array

def make_label_predicts(filename,model,thres=0.5):
    # labs = {0:'dent', 1:'glass_shatter',2: 'head_lamp',3: 'scratch', 4:'tail_lamp',5: 'unknown'}
    labs = {0:'unknown',
 1:'dent',  
 2:'scratch',
 3:'glass_shatter',
 4:'head_lamp',
 5:'tail_lamp'}
    img = cv2.imread(f"{filename}")
    if img is not None:
        # img = load_img(filename, target_size=(224, 224))
        # x = img_to_array(img) /255.0
        # x = np.expand_dims(x, axis=0)
        # x = preprocess_input(x)
        img = cv2.resize(img,(224, 224),interpolation=cv2.INTER_AREA)
        img = np.array(img) / 225.0
        img = img.reshape(-1,224,224,3)
        pred = np.squeeze(model.predict(img,verbose=0))
        # pred = np.squeeze(model.predict(x,verbose=0))
        # print('Predicted:', model.predict(img,verbose=0)) #decode_predictions(pred)[0])
        labels = []
        for i in range(len(pred)):
            if pred[i]>=thres:
                labels.append(labs[i])
        if len(labels)==0:
            labels.append('unknown')
        return labels

## metric calculation

In [19]:
mod = keras.models.load_model("./improvements1_1/weights-improvement-best_model.hdf5",custom_objects={'fbeta':fbeta})
predictions = {}
files = glob.glob('./Train/*')
c = 0
for f in files:
    fname = f.split('\\')[-1]
    predictions[f"image/{fname}"] = make_label_predicts(f"./Train/{fname}",mod,thres=0.6)

sub1 = pd.DataFrame({'image':predictions.keys(),'class':predictions.values()})
labeldf = pd.read_csv('./trainlabels.csv')
hash_map  = dict(labeldf[['image','class']].values)
actual = sub1['image'].apply(lambda x: [i.strip() for i in hash_map[x].split(',')])

mlb = MultiLabelBinarizer()
actuals = mlb.fit_transform(actual)
preds = mlb.transform(sub1['class'])
print("f1-score:",f1_score(actuals,preds,average='macro'))
from sklearn.metrics import accuracy_score
print("accuracy_score: ",accuracy_score(actuals,preds))
from sklearn.metrics import jaccard_score
print("jaccard_score: ",jaccard_score(actuals,preds,average='macro'))
from sklearn.metrics import fbeta_score
# pred_one =  np.asarray([np.ones(actuals.shape[1]) for _ in range(actuals.shape[0])])
print("beta2:",fbeta_score(actuals,preds,beta=2,average='samples'))
from sklearn.metrics import multilabel_confusion_matrix
print(multilabel_confusion_matrix(actuals,preds))

f1-score: 0.8915681024146647
accuracy_score:  0.83645443196005
jaccard_score:  0.8060925046257145
beta2: 0.8977177407352604
[[[503  15]
  [ 32 251]]

 [[657  12]
  [ 12 120]]

 [[613  20]
  [ 10 158]]

 [[502  40]
  [ 14 245]]

 [[680  11]
  [ 27  83]]

 [[590  26]
  [ 13 172]]]


##### moitoring
Baseline model
All ones:
0.5612645781185107

model valid:

Thresh: 0.3
beta2: 0.24638068678606243


Thresh: 0.5
beta2: 0.3153047127490837

Thresh: 0.6
beta2: 0.58 

In [21]:
mod = keras.models.load_model("./improvements1_1/weights-improvement-best_model.hdf5",custom_objects={'fbeta':fbeta})
predictions = {}
files = glob.glob('./Test/*')
c = 0
for f in files:
    fname = f.split('\\')[-1]
    predictions[f"image/{fname}"] = make_label_predicts(f"./Test/{fname}",mod,thres=0.6)

sub1 = pd.DataFrame({'image':predictions.keys(),'class':predictions.values()})

In [22]:
sub1.to_csv("../submissions/detectlabels.csv",index=False)

# Extent Of damage
* same steps as above.
* used same model architecture and its pre-trained weights for detecting extent of damage.

In [23]:
df = pd.read_csv("newtrainlabels.csv")
df['image'] = df['image'].apply(lambda x: x.split('/')[-1])
x = df.drop("extent_of_damage",axis=1)
y = df['extent_of_damage'].astype(str)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,stratify=y)
train = pd.concat([x_train,y_train],axis=1)
valid = pd.concat([x_test,y_test],axis=1)

In [24]:

def load_dataset(dir_path,train,valid):

    img_gen1 = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        ) 

    train_ds = img_gen1.flow_from_dataframe(
        dataframe=train,
        directory=dir_path,
        x_col="image",
        y_col="extent_of_damage",
        batch_size=12,
        seed=0,
        shuffle=True,
        class_mode="categorical",
        classes=["0","1","2","3"],
        target_size=(224,224)) # set as training data

    img_gen2 = ImageDataGenerator(rescale=1./255,
        ) 
    val_ds = img_gen2.flow_from_dataframe(
        dataframe=valid,
        directory=dir_path,
        x_col="image",
        y_col="extent_of_damage",
        batch_size=12,
        seed=0,
        shuffle=True,
        class_mode="categorical",
        classes=["0","1","2","3"],
        target_size=(224,224),) # set as validation data

    return train_ds, val_ds

In [77]:
train_ds, val_ds= load_dataset('./Train Copy',train.sample(train.shape[0]).reset_index(),valid.sample(valid.shape[0]).reset_index())

Found 864 validated image filenames belonging to 4 classes.
Found 217 validated image filenames belonging to 4 classes.


c:\projects\contelligenz\venv\lib\site-packages\keras\preprocessing\image.py:991: UserWarning: Found 2 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  n_invalid, x_col))


In [78]:
train_ds.next()[1]

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [79]:
train_ds.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3}

In [80]:
# def define_model(in_shape=(224, 224, 3), out_shape=4):
#   model = Sequential()
#   model.add(Conv2D(32, (3, 3), activation='relu',  padding='same', input_shape=in_shape))
#   model.add(Conv2D(32, (3, 3), activation='relu',  padding='same'))
#   model.add(MaxPooling2D((2, 2)))
#   model.add(Conv2D(64, (3, 3), activation='relu',  padding='same'))
#   model.add(Dropout(0.2))
#   model.add(Conv2D(64, (3, 3), activation='relu',  padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
#   model.add(MaxPooling2D((2, 2)))
#   model.add(Conv2D(128, (3, 3), activation='relu',  padding='same'))
#   model.add(Dropout(0.2))
#   model.add(Conv2D(128, (3, 3), activation='relu',  padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
#   model.add(MaxPooling2D((2, 2)))
#   model.add(Flatten())
#   model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
#   model.add(Dense(out_shape, activation='softmax'))
#   return model
def model_detect(num_expressions=6):
    model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3),
    pooling='max',
    classes=num_expressions,
)
    regularizer = tf.keras.regularizers.l2(0.1)
    for layer in model.layers[:-5]:
      layer.trainable= False
    for layer in model.layers:
      if isinstance(layer, tf.keras.layers.Conv2D):
        print('Adding regularizer to layer {}'.format(layer.name))
        layer.kernel_regularizer = regularizer
    # model.get_layer('block5_conv1').trainable = True
    # model.get_layer('block5_conv2').trainable = True
    # model.get_layer('block5_conv3').trainable = True
    # model.get_layer('block5_pool').trainable = True
    flat1 = Flatten()(model.layers[-1].output)
    layer1 = Dropout(0.4)(flat1)
    layer2 = Dense(128, activation='relu', kernel_initializer='he_uniform')(layer1)
    # layer3 = Dense(64, activation='relu', kernel_initializer='he_uniform')(layer2)
    output = Dense(6, activation='sigmoid')(layer2)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    return model

In [81]:
model1 = keras.models.load_model("./improvements1_1/weights-improvement-best_model.hdf5",custom_objects={'fbeta':fbeta})
# model2 = model_detect()
model2 =Sequential()
for layer in model1.layers:
   model2.add(layer)
# Freeze the layers 
for layer in model2.layers:
    layer.trainable = False
model2.add(Dense(4, activation='softmax'))
model2.compile(optimizer=Adam(0.001), loss=CategoricalCrossentropy(from_logits=False),metrics=keras.metrics.Precision(name='precision'))
filepath="./improvements2_1/weights-improvement-best_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_precision', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

hist = model2.fit(
    train_ds,
    steps_per_epoch = train_ds.samples // 12,
    validation_data = val_ds, 
    validation_steps = val_ds.samples // 12,
    epochs = 25,
    callbacks=callbacks_list
)

Epoch 1/25
72/72 [==============================] - ETA: 0s - loss: 161.8868 - precision: 0.0000e+00
Epoch 1: val_precision improved from -inf to 0.00000, saving model to ./improvements2_1\weights-improvement-best_model.hdf5
72/72 [==============================] - 25s 315ms/step - loss: 161.8868 - precision: 0.0000e+00 - val_loss: 161.8698 - val_precision: 0.0000e+00
Epoch 2/25
72/72 [==============================] - ETA: 0s - loss: 161.8032 - precision: 0.0000e+00
Epoch 2: val_precision did not improve from 0.00000
72/72 [==============================] - 22s 299ms/step - loss: 161.8032 - precision: 0.0000e+00 - val_loss: 161.7916 - val_precision: 0.0000e+00
Epoch 3/25
72/72 [==============================] - ETA: 0s - loss: 161.7328 - precision: 0.0000e+00
Epoch 3: val_precision did not improve from 0.00000
72/72 [==============================] - 22s 298ms/step - loss: 161.7328 - precision: 0.0000e+00 - val_loss: 161.7299 - val_precision: 0.0000e+00
Epoch 4/25
72/72 [=============

In [82]:
from keras.utils import load_img, img_to_array

def make_label_predicts(filename,model,thres=0.5):
    labs = {0:'0',
 1:'1',  
 2:'2',
 3:'3',
 4:'4'}
    img = cv2.imread(f"{filename}")
    if img is not None:
        img = cv2.resize(img,(224, 224),interpolation=cv2.INTER_AREA)
        img = np.array(img) / 225.0
        img = img.reshape(-1,224,224,3)
        pred = np.squeeze(model.predict(img,verbose=0))
        return labs[np.argmax(pred)]

In [83]:
model2 = keras.models.load_model("./improvements2_1/weights-improvement-best_model.hdf5")

In [84]:
predictions = {}
files = glob.glob('./Train/*')
c = 0
for f in files:
    fname = f.split('\\')[-1]
    predictions[f"image/{fname}"] = make_label_predicts(f"./Train/{fname}",model2,thres=0.6)

sub1 = pd.DataFrame({'image':predictions.keys(),'extent_of_damage':predictions.values()})
labeldf = pd.read_csv('./trainlabels.csv')
hash_map  = dict(labeldf[['image','extent_of_damage']].values)
actual = sub1['image'].apply(lambda x:str(hash_map[x]))

# mlb = MultiLabelBinarizer()
# actuals = mlb.fit_transform(actual)
# preds = mlb.transform(sub1['class'])
print("f1-score:",f1_score(actual,sub1['extent_of_damage'],average='macro'))
from sklearn.metrics import accuracy_score
print("accuracy_score: ",accuracy_score(actual,sub1['extent_of_damage']))
from sklearn.metrics import jaccard_score
print("jaccard_score: ",jaccard_score(actual,sub1['extent_of_damage'],average='macro'))
from sklearn.metrics import fbeta_score
# pred_one =  np.asarray([np.ones(actuals.shape[1]) for _ in range(actuals.shape[0])])

f1-score: 0.5973886229594672
accuracy_score:  0.8002496878901373
jaccard_score:  0.5132688939208994


f1-score: 0.6276695790006568
accuracy_score:  0.7116104868913857
jaccard_score:  0.4674106760167673

In [85]:
# mod = keras.models.load_model("./improvements2_2/weights-improvement-best_model.hdf5",custom_objects={'fbeta':fbeta})
predictions = {}
files = glob.glob('./Test/*')
c = 0
for f in files:
    fname = f.split('\\')[-1]
    predictions[f"image/{fname}"] = make_label_predicts(f"./Test/{fname}",model2,thres=0.6)

sub1 = pd.DataFrame({'image':predictions.keys(),'extent_of_damage':predictions.values()})


In [86]:
sub1.to_csv('../submissions/extentlabels.csv',index=False)

# Final Prediction

In [88]:
damagelabels = pd.read_csv("../submissions/detectlabels.csv")
extentlabels = pd.read_csv("../submissions/extentlabels.csv")

In [89]:
damagelabels.head()

,image,class
0,image/1000.jpeg,['dent']
1,image/1001.jpeg,['scratch']
2,image/1002.jpeg,['glass_shatter']
3,image/1003.jpeg,['unknown']
4,image/1004.jpeg,['tail_lamp']


In [90]:
extentlabels.head()

,image,extent_of_damage
0,image/1000.jpeg,1
1,image/1001.jpeg,1
2,image/1002.jpeg,1
3,image/1003.jpeg,0
4,image/1004.jpeg,1


In [91]:
finalsub = pd.merge(damagelabels,extentlabels,on='image')

In [92]:
finalsub.isnull().sum()

image               0
class               0
extent_of_damage    0
dtype: int64

In [93]:
finalsub['dent'] = finalsub['class'].apply(lambda x: 1 if 'dent' in x else 0)
finalsub['glass_shatter'] = finalsub['class'].apply(lambda x: 1 if 'glass_shatter' in x else 0)
finalsub['head_lamp'] = finalsub['class'].apply(lambda x: 1 if 'head_lamp' in x else 0)
finalsub['scratch'] = finalsub['class'].apply(lambda x: 1 if 'scratch' in x else 0)
finalsub['tail_lamp'] = finalsub['class'].apply(lambda x: 1 if 'tail_lamp' in x else 0)
finalsub['unknown'] = finalsub['class'].apply(lambda x: 1 if 'unknown' in x else 0)

In [94]:
finalsub.head()

,image,class,extent_of_damage,dent,glass_shatter,head_lamp,scratch,tail_lamp,unknown
0,image/1000.jpeg,['dent'],1,1,0,0,0,0,0
1,image/1001.jpeg,['scratch'],1,0,0,0,1,0,0
2,image/1002.jpeg,['glass_shatter'],1,0,1,0,0,0,0
3,image/1003.jpeg,['unknown'],0,0,0,0,0,0,1
4,image/1004.jpeg,['tail_lamp'],1,0,0,0,0,1,0


In [95]:
ext = finalsub['extent_of_damage']
sub = finalsub.drop(['class','extent_of_damage'],axis=1)
sub['extent_of_damage'] = ext
f = sub['image'].apply(lambda x: int(x.split('/')[-1].split('.')[0]))
f.min(), f.max()

(801, 1100)

In [96]:
sub['image_num'] = sub['image'].apply(lambda x: int(x.split('/')[-1].split('.')[0]))
sub = sub.sort_values('image_num',ascending=True)
sub = sub.drop(['image_num'],axis=1)

In [103]:
sub.head()

,image,dent,glass_shatter,head_lamp,scratch,tail_lamp,unknown,extent_of_damage
101,image/801.jpeg,0,0,0,0,1,0,1
102,image/802.jpeg,1,0,1,1,0,0,1
103,image/803.jpeg,0,0,0,0,0,1,0
104,image/804.jpeg,1,0,0,0,0,0,1
105,image/805.jpeg,1,0,0,1,0,0,1


In [105]:
sub.to_excel("../submissions/prediction_atufa.xlsx",index=False)

# Helper Function
1. Load imports
2. Run cells of this section
3. run the function predict_image(<image_path>,<model_detect_path>,<model_extent_path>)
Note: 
* detectweights-model is model_detect_path
* extentweights-model is model_extent model_extent_path

In [99]:
def make_label_predicts(filename,model,thres=0.5):
    labs = {0:'0',
 1:'1',  
 2:'2',
 3:'3',
 4:'4'}
    img = cv2.imread(f"{filename}")
    if img is not None:
        img = cv2.resize(img,(224, 224),interpolation=cv2.INTER_AREA)
        img = np.array(img) / 225.0
        img = img.reshape(-1,224,224,3)
        pred = np.squeeze(model.predict(img,verbose=0))
        return labs[np.argmax(pred)]

In [100]:
from keras.utils import load_img, img_to_array

def make_label_predicts(filename,model,thres=0.5):
    labs = {0:'unknown',
 1:'dent',  
 2:'scratch',
 3:'glass_shatter',
 4:'head_lamp',
 5:'tail_lamp'}
    img = cv2.imread(f"{filename}")
    if img is not None:
        img = cv2.resize(img,(224, 224),interpolation=cv2.INTER_AREA)
        img = np.array(img) / 225.0
        img = img.reshape(-1,224,224,3)
        pred = np.squeeze(model.predict(img,verbose=0))
        labels = []
        for i in range(len(pred)):
            if pred[i]>=thres:
                labels.append(labs[i])
        if len(labels)==0:
            labels.append('unknown')
        return labels

In [101]:
from keras import backend
 
# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

def damage_detect(image_path,model_damage,thresh=0.6):
        labs = {0:'unknown',
                1:'dent',  
                2:'scratch',
                3:'glass_shatter',
                4:'head_lamp',
                5:'tail_lamp'}
        img = cv2.imread(f"{image_path}")
        labels=[]
        if img is not None:
                img = cv2.resize(img,(224, 224),interpolation=cv2.INTER_AREA)
                img = np.array(img) / 225.0
                img = img.reshape(-1,224,224,3)
                pred = np.squeeze(model_damage.predict(img,verbose=0))
                labels = []
                for i in range(len(pred)):
                        if pred[i]>=thresh:
                                labels.append(labs[i])
                if len(labels)==0:
                        labels.append('unknown')
        else:
                return "Image cannot be read!"
        return labels

def extent_detect(image_path,model_extent):
        labs = {0:'0',
                1:'1',  
                2:'2',
                3:'3',
                4:'4'}
        img = cv2.imread(f"{image_path}")
        if img is not None:
                img = cv2.resize(img,(224, 224),interpolation=cv2.INTER_AREA)
                img = np.array(img) / 225.0
                img = img.reshape(-1,224,224,3)
                pred = np.squeeze(model_extent.predict(img,verbose=0))
                return labs[np.argmax(pred)]

def predict_image(image_path,model_detect_path,model_extent_path):
        model_detect = keras.models.load_model(model_detect_path,custom_objects={'fbeta':fbeta})
        labels = damage_detect(image_path,model_detect)
        model_extent = keras.models.load_model(model_extent_path,custom_objects={'fbeta':fbeta})
        extent = extent_detect(image_path,model_extent)
        dent = 1 if 'dent' in labels else 0
        glass_shatter = 1 if 'glass_shatter' in labels else 0
        head_lamp = 1 if 'head_lamp' in labels else 0
        scratch = 1 if 'scratch' in labels else 0
        tail_lamp = 1 if 'tail_lamp' in labels else 0
        unknown = 1 if 'unknown' in labels else 0
        return {'image':image_path,'dent':dent,'glass_shatter':glass_shatter,'head_lamp':head_lamp,'scratch':scratch,'tail_lamp':tail_lamp,'unknown':unknown,'extent_of_damage':extent}


### Example

In [106]:
predict_image('./Train/0.jpeg','../submissions/detectweights-model.hdf5','../submissions/extentweights-model.hdf5')

{'image': './Train/0.jpeg',
 'dent': 1,
 'glass_shatter': 0,
 'head_lamp': 1,
 'scratch': 0,
 'tail_lamp': 0,
 'unknown': 0,
 'extent_of_damage': '1'}